In [1]:
input = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""

input = open("inputs/6").read()

In [2]:
import numpy as np


def parse_board(input):
    return np.array(list(map(list, input.splitlines())))


board = parse_board(input)
board

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '#', '.', ..., '#', '.', '.'],
       ['.', '#', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

In [3]:
start_pos = np.where(board == "^")
start_tuple = (int(start_pos[0][0]), int(start_pos[1][0]))
start_tuple

(80, 79)

In [4]:
# replace with empty space
board[start_tuple] = "."

In [5]:
board

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '#', '.', ..., '#', '.', '.'],
       ['.', '#', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

In [6]:
def add_tuple(t1, t2):
    return tuple(x + y for x, y in zip(t1, t2))


# def generate_path(start, dir, num_steps):
#     location = start
#     yield location
#     for _ in range(num_steps - 1):
#         location = add_tuple(location, dir)
#         yield location


def in_bounds(location, bounds):
    return all(0 <= x < y for x, y in zip(location, bounds))


# def get_values(start, dir, steps, board):
#     return [
#         board[location]
#         for location in generate_path(start, dir, steps)
#         if in_bounds(location, board.shape)
#     ]

In [7]:
from collections import deque

dirs = deque(
    [
        (-1, 0),  # up
        (0, 1),  # right
        (1, 0),  # down
        (0, -1),  # left
    ]
)

visited = set()

# in my board and the test board it always starts facing up
current_location = start_tuple
visited.add(current_location)

while True:
    current_dir = dirs[0]
    next_location = add_tuple(current_location, current_dir)
    # print(next_location)

    if not in_bounds(next_location, board.shape):
        # left map, we're done
        break

    if board[next_location] == ".":
        current_location = next_location
        visited.add(current_location)
    else:
        # turn right
        dirs.rotate(-1)

len(visited)

5131

In [8]:
# pt2
free_spaces = {(int(x), int(y)) for x, y in zip(*np.where(board == "."))} - {
    start_tuple
}
len(free_spaces)

16087

In [10]:
# I'm aware that the correct way to do this problem is to make a graph with 4*num_free_spaces nodes (one for each direction at each free space) and then look at the paths in the graph, but I'm too lazy to implement that
# for pt2 you just modify the graph slightly and then look for a cycle

In [9]:
from collections import deque
from tqdm import tqdm

start_dirs = deque(
    [
        (-1, 0),  # up
        (0, 1),  # right
        (1, 0),  # down
        (0, -1),  # left
    ]
)

num_loops = 0
for free_space in tqdm(free_spaces):
    new_board = board.copy()
    new_board[free_space] = "#"

    dirs = start_dirs.copy()

    visited = set()
    visited_with_dir = set()

    # in my board and the test board it always starts facing up
    current_location = start_tuple
    visited.add(current_location)

    while True:
        current_dir = dirs[0]
        next_location = add_tuple(current_location, current_dir)

        if not in_bounds(next_location, new_board.shape):
            # left map, we're done
            break

        if (next_location, current_dir) in visited_with_dir:
            num_loops += 1
            break

        if new_board[next_location] == ".":
            current_location = next_location
            visited.add(current_location)
            visited_with_dir.add((current_location, current_dir))
        else:
            # turn right
            dirs.rotate(-1)

num_loops


100%|██████████| 16087/16087 [02:13<00:00, 120.14it/s]


1784